In [4]:
import pyddm as ddm
from pyddm import Model, Fittable, Fitted, Sample
from pyddm.models import Bound, Drift, Noise, Overlay, LossFunction, ICPointSourceCenter


In [8]:
pip install pyddm --upgrade

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.1.2 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [7]:
# Definir el drift combinado para respuestas correctas e incorrectas
class DriftCombined(ddm.models.Drift):
    name = "Combined drift for correct and incorrect responses"
    required_parameters = ["drift_correct", "drift_incorrect"]
    
    def get_drift(self, conditions, **kwargs):
        return self.drift_correct - self.drift_incorrect

# Definir los límites
class BoundFixed(ddm.models.Bound):
    name = "Fixed boundary"
    required_parameters = ["B"]
    
    def get_bound(self, t, conditions, **kwargs):
        return self.B

# Definir el ruido
class NoiseConstant(ddm.models.Noise):
    name = "Constant noise"
    required_parameters = ["noise"]
    
    def get_noise(self, t, conditions, **kwargs):
        return self.noise

# Definir el modelo con dos acumuladores
class ModelTwoAccumulators(ddm.Model):
    def __init__(self, drift_correct, drift_incorrect, bound, noise):
        drift = DriftCombined(drift_correct=drift_correct, drift_incorrect=drift_incorrect)
        bound = BoundFixed(B=bound)
        noise = NoiseConstant(noise=noise)
        overlay = ddm.models.OverlayNonDecision(nondectime=0.3)
        ic = ICPointSourceCenter()
        super().__init__(drift=drift, bound=bound, noise=noise, overlay=overlay, ic=ic)

# Crear una instancia del modelo
model = ModelTwoAccumulators(drift_correct=0.5, drift_incorrect=0.3, bound=1.0, noise=1.0)

# Simular datos
samples = model.simulate_trial(n=1000)

# Calcular confianza
samples["confidence"] = samples.corr * (samples.rt - 0.3) / samples.rt

# Mostrar algunos resultados
print(samples[:10])



TypeError: Model.__init__() got an unexpected keyword argument 'ic'